### Running in Docker container on Ostrich

#### Started Docker container with the following command:

```docker run - p 8888:8888 -v /Users/sam/data/:/data -v /Users/sam/owl_home/:/owl_home -v /Users/sam/owl_web/:owl_web -v /Users/sam/gitrepos/LabDocs/jupyter_nbs/sam/:/jupyter_nbs -it 0ba43904567e```

The command allows access to Jupyter Notebook over port 8888 and makes my Jupyter Notebook GitHub repo and my data files on Owl/home and Owl/web accessible to the Docker container.

Once the container was started, started Jupyter Notebook with the following command inside the Docker container:

```jupyter notebook```

This is configured in the Docker container to launch a Jupyter Notebook without a browser on port 8888.

The Docker container is running on an image created from this [Dockerfile (Git commit 65ab831)](https://github.com/sr320/LabDocs/blob/65ab8317556e01c8b3ae3ca260119b1eb163ed44/code/dockerfiles/Dockerfile.bio)

In [1]:
%%bash
date

Thu Nov 17 22:30:49 UTC 2016


### Check computer specs

In [2]:
%%bash
hostname

65e23f54bcc9


In [3]:
%%bash
lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                8
On-line CPU(s) list:   0-7
Thread(s) per core:    1
Core(s) per socket:    8
Socket(s):             1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 26
Model name:            Intel(R) Xeon(R) CPU           E5520  @ 2.27GHz
Stepping:              5
CPU MHz:               2260.998
BogoMIPS:              4521.99
Hypervisor vendor:     KVM
Virtualization type:   full
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              8192K


### View PyRad output files

In [4]:
%%bash
ls -l /root/owl_web/Athaliana/20160715_pyrad_outfiles

total 1908968
-rw-rw-rw- 1 srlab staff 260547113 Aug 16 15:46 oly_gbs_pyrad.alleles
-rw-rw-rw- 1 srlab staff 239419515 Aug 16 15:46 oly_gbs_pyrad.excluded_loci
-rw-rw-rw- 1 srlab staff 126920538 Aug 16 15:46 oly_gbs_pyrad.gphocs
-rw-rw-rw- 1 srlab staff 133378169 Aug 16 15:46 oly_gbs_pyrad.loci
-rw-rw-rw- 1 srlab staff     86991 Aug 16 15:47 oly_gbs_pyrad.migrate
-rw-rw-rw- 1 srlab staff 571222263 Aug 16 15:47 oly_gbs_pyrad.nex
-rw-rw-rw- 1 srlab staff 463115051 Aug 16 15:47 oly_gbs_pyrad.phy
-rw-rw-rw- 1 srlab staff   1266466 Aug 16 15:46 oly_gbs_pyrad.phy.partitions
-rw-rw-rw- 1 srlab staff  26795940 Aug 16 15:46 oly_gbs_pyrad.snps
-rw-rw-rw- 1 srlab staff  11519817 Aug 16 15:46 oly_gbs_pyrad.snps.geno
-rwxrwxrwx 1 srlab staff  13474538 Aug 16 22:20 oly_gbs_pyrad.str
-rw-rw-rw- 1 srlab staff     30531 Aug 16 15:46 oly_gbs_pyrad.treemix.gz
-rw-rw-rw- 1 srlab staff   2417961 Aug 16 15:46 oly_gbs_pyrad.unlinked_snps
-rw-rw-rw- 1 srlab staff   2441102 Aug 16 15:46 oly_gbs_pyrad.usnps.gen

### Make working directory & move PyRad VCF file to working directory

In [5]:
%%bash
mkdir /root/data/20161117_oly_gbs_vcf_analysis

In [6]:
%%bash
time cp -avr /root/owl_web/Athaliana/20160715_pyrad_outfiles/oly_gbs_pyrad.vcf /root/data/20161117_oly_gbs_vcf_analysis/

'/root/owl_web/Athaliana/20160715_pyrad_outfiles/oly_gbs_pyrad.vcf' -> '/root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf'



real	0m2.747s
user	0m0.000s
sys	0m0.410s


### View the VCF header info

In [7]:
%%bash
head -12 /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf

##fileformat=VCFv4.1
##fileDate=20160725
##source=pyRAD.v.3.0.66
##reference=common_allele_at_each_locus
##INFO=<ID=NS,Number=1,Type=Integer,Description="Number of Samples With Data">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Total Depth">
##INFO=<ID=AF,Number=A,Type=Float,Description="Allele Frequency">
##INFO=<ID=AA,Number=1,Type=String,Description="Ancestral Allele">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Read Depth">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO    	FORMAT	1HL_10A.assembled	1HL_11A.assembled	1HL_12A.assembled	1HL_13A.assembled	1HL_14A.assembled	1HL_15A.assembled	1HL_16A.assembled	1HL_17A.assembled	1HL_19A.assembled	1HL_1A.assembled	1HL_20A.assembled	1HL_21A.assembled	1HL_22A.assembled	1HL_23A.assembled	1HL_24A.assembled	1HL_25A.assembled	1HL_26A.assembled	1HL_27A.assembled	1HL_28A.assembled	1HL_29A.assembled	1HL_2A.assembled

### View the header again and perform line count

#### This is done to show that the last line of the header (with all of the sample names) is truly a single line

In [8]:
%%bash
head -12 /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf | wc -l

12


### Create population list files - necessary for subsequent vcftools commands

#### vcftools requires separate population files that list the sample IDs for each individual within each population.  The command below uses a for loop to do the following:

#### ```$(awk 'NR == 12 {print $0}'``` - This uses ```awk``` to process only the 12th line (```NR == 12```) of the file and prints all the fields (```{print $0}```) on that line.

#### ```if [[ "$i" =~ 1HL.* ]] then echo "$i" >> /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop``` - A set of if/elif statements that evaluate each field from the awk command and determines if they match a pattern (```[[ "$i" =~ 1HL.* ]]```. If they match that pattern, the value currently stored in ```$i``` is written to the appropriate population file.

In [24]:
%%bash
time for i in $(awk 'NR == 12 {print $0}' /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf)
do
    if [[ "$i" =~ 1HL.* ]]
        then echo "$i" >> /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop
        elif [[ "$i" =~ 1NF.* ]]
            then echo "$i" >> /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop
        elif [[ "$i" =~ 1SN.* ]]
            then echo "$i" >> /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop
    fi
done


real	0m2.496s
user	0m0.050s
sys	0m0.900s


### Verify population files contents

In [25]:
%%bash
cat /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop

1HL_10A.assembled
1HL_11A.assembled
1HL_12A.assembled
1HL_13A.assembled
1HL_14A.assembled
1HL_15A.assembled
1HL_16A.assembled
1HL_17A.assembled
1HL_19A.assembled
1HL_1A.assembled
1HL_20A.assembled
1HL_21A.assembled
1HL_22A.assembled
1HL_23A.assembled
1HL_24A.assembled
1HL_25A.assembled
1HL_26A.assembled
1HL_27A.assembled
1HL_28A.assembled
1HL_29A.assembled
1HL_2A.assembled
1HL_31A.assembled
1HL_33A.assembled
1HL_34A.assembled
1HL_35A.assembled
1HL_3A.assembled
1HL_4A.assembled
1HL_5A.assembled
1HL_6A.assembled
1HL_7A.assembled
1HL_8A.assembled
1HL_9A.assembled


In [26]:
%%bash
cat /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop

1NF_10A.assembled
1NF_11A.assembled
1NF_12A.assembled
1NF_13A.assembled
1NF_14A.assembled
1NF_15A.assembled
1NF_16A.assembled
1NF_17A.assembled
1NF_18A.assembled
1NF_19A.assembled
1NF_1A.assembled
1NF_20A.assembled
1NF_21A.assembled
1NF_22A.assembled
1NF_23A.assembled
1NF_24A.assembled
1NF_25A.assembled
1NF_26A.assembled
1NF_27A.assembled
1NF_28A.assembled
1NF_29A.assembled
1NF_2A.assembled
1NF_30A.assembled
1NF_31A.assembled
1NF_32A.assembled
1NF_33A.assembled
1NF_4A.assembled
1NF_5A.assembled
1NF_6A.assembled
1NF_7A.assembled
1NF_8A.assembled
1NF_9A.assembled


In [27]:
%%bash
cat /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop

1SN_10A.assembled
1SN_11A.assembled
1SN_12A.assembled
1SN_13A.assembled
1SN_14A.assembled
1SN_15A.assembled
1SN_16A.assembled
1SN_17A.assembled
1SN_18A.assembled
1SN_19A.assembled
1SN_1A.assembled
1SN_20A.assembled
1SN_21A.assembled
1SN_22A.assembled
1SN_23A.assembled
1SN_24A.assembled
1SN_25A.assembled
1SN_26A.assembled
1SN_27A.assembled
1SN_28A.assembled
1SN_29A.assembled
1SN_2A.assembled
1SN_30A.assembled
1SN_31A.assembled
1SN_32A.assembled
1SN_3A.assembled
1SN_4A.assembled
1SN_5A.assembled
1SN_6A.assembled
1SN_7A.assembled
1SN_8A.assembled
1SN_9A.assembled


### Verify each population file has expected number of individuals (32)

In [29]:
%%bash
for i in /root/data/20161117_oly_gbs_vcf_analysis/*.pop
do
    wc -l "$i"
done

32 /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop
32 /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop
32 /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop


### Verify instllation of vcftools

In [30]:
%%bash
which vcftools

/usr/local/bin/vcftools


### Determine Fst values between all population combinations

#### 1HL vs. 1NF

In [31]:
%%bash
time vcftools --vcf /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop

mv out.weir.fst /root/data/20161117_oly_gbs_vcf_analysis/1HL_NF.fst


VCFtools - 0.1.14
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop

Eighth Header entry should be INFO: INFO    
Keeping individuals in 'keep' list
After filtering, kept 64 out of 96 Individuals
Outputting Weir and Cockerham Fst estimates.
Weir and Cockerham mean Fst estimate: 0.0029054
Weir and Cockerham weighted Fst estimate: 0.14582
After filtering, kept 120130 out of a possible 120130 Sites
Run Time = 25.00 seconds

real	0m24.933s
user	0m0.760s
sys	0m13.630s


#### 1HL vs. 1SN

In [32]:
%%bash
time vcftools --vcf /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop

mv out.weir.fst /root/data/20161117_oly_gbs_vcf_analysis/1HL_SN.fst


VCFtools - 0.1.14
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop

Eighth Header entry should be INFO: INFO    
Keeping individuals in 'keep' list
After filtering, kept 64 out of 96 Individuals
Outputting Weir and Cockerham Fst estimates.
Weir and Cockerham mean Fst estimate: 0.018513
Weir and Cockerham weighted Fst estimate: 0.18011
After filtering, kept 120130 out of a possible 120130 Sites
Run Time = 24.00 seconds

real	0m24.362s
user	0m0.800s
sys	0m13.780s


#### 1NF vs 1SN

In [33]:
%%bash
time vcftools --vcf /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop

mv out.weir.fst /root/data/20161117_oly_gbs_vcf_analysis/1NF_1SN.fst


VCFtools - 0.1.14
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop

Eighth Header entry should be INFO: INFO    
Keeping individuals in 'keep' list
After filtering, kept 64 out of 96 Individuals
Outputting Weir and Cockerham Fst estimates.
Weir and Cockerham mean Fst estimate: -0.010855
Weir and Cockerham weighted Fst estimate: 0.10355
After filtering, kept 120130 out of a possible 120130 Sites
Run Time = 27.00 seconds

real	0m26.961s
user	0m1.890s
sys	0m12.430s


#### 1HL vs. 1NF vs. 1SN

In [34]:
%%bash
time vcftools --vcf /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop \
--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop

mv out.weir.fst /root/data/20161117_oly_gbs_vcf_analysis/1HL_1NF_1SN.fst


VCFtools - 0.1.14
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /root/data/20161117_oly_gbs_vcf_analysis/oly_gbs_pyrad.vcf
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop
	--weir-fst-pop /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1HL.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1NF.pop
	--keep /root/data/20161117_oly_gbs_vcf_analysis/1SN.pop

Eighth Header entry should be INFO: INFO    
Keeping individuals in 'keep' list
After filtering, kept 96 out of 96 Individuals
Outputting Weir and Cockerham Fst estimates.
Weir and Cockerham mean Fst estimate: 0.048516
Weir and Cockerham weighted Fst estimate: 0.13739
After filtering, kept 120130 out of a possible 120130 Sites
Run Time = 25.00 seconds

real	0m24.840s
user	0m1.080s
sys	0m13.720s
